Pour obtenir le df, lancer depuis un terminal la commande suite :
python scripts/compute_nli.py data/01_251021_Notations200.csv data/01_251021_Notations200_with_nli.csv

In [14]:
# Corrélation de Pearson entre score humain et score nli entailment/contradiction/entailment-contradiction
df["nli_entailment_minus_contradiction"] = (
    df["nli_entailment"] - df["nli_contradiction"]
)

correlation_entailment = df["score_humain"].corr(df["nli_entailment"])
print(f"Corrélation entre le score humain moyen et le score NLI entailment : {correlation_entailment:.2f}")

correlation_contradiction = df["score_humain"].corr(df["nli_1_minus_contradiction"])
print(f"Corrélation entre le score humain moyen et le score NLI contradiction : {correlation_contradiction:.2f}")

correlation_ent_minus_contra = df["score_humain"].corr(
    df["nli_entailment_minus_contradiction"]
)
print(f"Corrélation entre le score humain moyen et le score NLI difference : {correlation_ent_minus_contra:.2f}")

Corrélation entre le score humain moyen et le score NLI entailment : 0.54
Corrélation entre le score humain moyen et le score NLI contradiction : 0.15
Corrélation entre le score humain moyen et le score NLI difference : 0.51


In [8]:
# Chargement des données
df = pd.read_csv("../data/01_251021_Notations200_with_nli.csv")

# Calcul du score humain moyen
df["score_humain"] = df[["Garance", "Matthias", "Yannis"]].mean(axis=1)

# Copie des colonnes d'hallucinations/d'idées invalides en version catégorielle
mapping = {0:"Non", 1:"Oui"}
df["pres_hallu"] = df["Hallucinations"].map(mapping)
df["pres_idees_inv"] = df["Idées_non_ind"].map(mapping)

# Nombre de tokens par contribution et par extractions (comptage simple par espaces)
df["nb_tokens_contrib"] = df["contribution"].apply(lambda x: len(str(x).split()))
df["nb_tokens_extraction"] = df["ideas_text"].apply(lambda x: len(str(x).split()))

# Répartition des contributions en 5 catégories équilibrées selon le nombre de tokens (basé sur les quantiles)
df["contrib_tokens_bins"] = pd.qcut(df["nb_tokens_contrib"], q=5, labels=[
    "Très court", "Court", "Moyen", "Long", "Très long"
])

# Autre options : catégories basées sur des seuils fixes
df["contrib_tokens_bins_fixe"] = pd.cut(df["nb_tokens_contrib"], bins=[0, 50, 100, 200, np.inf], labels=[
    "Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"
])

In [9]:
fig = px.density_contour(
    df, x="nli_entailment", y="score_humain",
    nbinsx=20, nbinsy=20
)

fig.update_traces(
    contours_coloring="fill",
    contours_showlabels=False,
    colorscale="Blues"
)

# Scatter plot
fig.add_scatter(
    x=df["nli_entailment"],
    y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate=
        "NLI (entailment) = %{x}<br>"
        "Score humain = %{y}<extra></extra>"
)

# Ligne de référence : y = 10x
fig.add_shape(
    type="line",
    x0=0, x1=1,
    y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)

fig.update_layout(
    title={"text": "<b>Score humain vs NLI (entailment)</b>"},
    xaxis_title="NLI — entailment",
    yaxis_title="Score humain",
    width=700, height=500,
    coloraxis_showscale=False
)

fig.show()

In [ ]:
fig = px.density_contour(
    df, x="nli_1_minus_contradiction", y="score_humain",
    nbinsx=20, nbinsy=20
)

fig.update_traces(
    contours_coloring="fill",
    contours_showlabels=False,
    colorscale="Blues"
)

# Scatter plot
fig.add_scatter(
    x=df["nli_1_minus_contradiction"],
    y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate=
        "NLI (1 - contradiction) = %{x}<br>"
        "Score humain = %{y}<extra></extra>"
)

# Ligne de référence : y = 10x
fig.add_shape(
    type="line",
    x0=0, x1=1,
    y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)

fig.update_layout(
    title={"text": "<b>Score humain vs NLI (1 - contradiction)</b>"},
    xaxis_title="NLI — 1 - contradiction",
    yaxis_title="Score humain",
    width=700, height=500,
    coloraxis_showscale=False
)

fig.show()

In [ ]:

fig = px.scatter(
    df, x="nli_entailment", y="score_humain",
    color="pres_hallu",
    color_discrete_sequence=["#ff6361", "#003f5c"]
)

# Référence: y = 10x (car x∈[0,1], y∈[0,10])
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)

fig.update_layout(
    title={"text": "<b>Score humain vs NLI (entailment)</b>"},
    xaxis_title="NLI — entailment",
    yaxis_title="Score humain",
    legend_title="Présence <br>d'hallucinations",
    width=700, height=500
)

fig.update_traces(
    hovertemplate=
        "NLI (entailment) = %{x}<br>"
        "Score humain = %{y}<extra></extra>"
)

fig.show()


In [ ]:
fig = px.scatter(
    df, x="nli_1_minus_contradiction", y="score_humain",
    color="pres_hallu",
    color_discrete_sequence=["#ff6361", "#003f5c"]
)

fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)

fig.update_layout(
    title={"text": "<b>Score humain vs NLI (1 - contradiction)</b>"},
    xaxis_title="NLI — 1 - contradiction",
    yaxis_title="Score humain",
    legend_title="Présence <br>d'hallucinations",
    width=700, height=500
)

fig.update_traces(
    hovertemplate=
        "NLI (1 - contradiction) = %{x}<br>"
        "Score humain = %{y}<extra></extra>"
)

fig.show()

In [4]:
df = pd.read_csv("../data/01_251021_Notations200_with_nli.csv")

In [5]:
df.head()

,authorId,contrib_index,contribution,ideas_text,C,n_ideas,len_contrib,len_ideas_text,Matthias,Yannis,Garance,Hallucinations,Idées_non_ind,score_humain,nli_entailment,nli_contradiction,nli_1_minus_contradiction
0,VXNlcjpmN2E5MjUwYS0xZTIzLTExZTktOTRkMi1mYTE2M2...,118,Supprimer la CSG sur le pensions et les rentes,[PARSE_FAIL] Colonnes manquantes: ['descriptio...,0.071738,1,46,79,0,0,0,NaN,NaN,0.0,0.194438,0.604606,0.395394
1,VXNlcjo2ZTgxMmU5ZS0xZTIzLTExZTktOTRkMi1mYTE2M2...,127,Je souhaite débattre du sujet suivant : Le car...,[PARSE_FAIL] Colonnes manquantes: ['descriptio...,0.022147,1,7557,79,0,0,0,NaN,NaN,0.0,0.412060,0.422971,0.577029
2,VXNlcjpkZTIzYzdjNC0xZDUyLTExZTktOTRkMi1mYTE2M2...,128,Question sans rapport avec le sujet.,[PARSE_FAIL] Colonnes manquantes: ['descriptio...,0.126939,1,36,79,0,0,0,NaN,NaN,0.0,0.250277,0.354189,0.645811
3,VXNlcjpkZTIzYzdjNC0xZDUyLTExZTktOTRkMi1mYTE2M2...,144,Question sans rapport avec le sujet.,[PARSE_FAIL] Colonnes manquantes: ['descriptio...,0.126939,1,36,79,0,0,0,NaN,NaN,0.0,0.250277,0.354189,0.645811
4,VXNlcjpkZTIzYzdjNC0xZDUyLTExZTktOTRkMi1mYTE2M2...,192,Question sans rapport avec le sujet.,[PARSE_FAIL] Colonnes manquantes: ['descriptio...,0.126939,1,36,79,0,0,0,NaN,NaN,0.0,0.250277,0.354189,0.645811
